In [14]:
import sys
import numpy as np
sys.path.append("..")
from fireballpy import Fireball
from ase import Atoms
from ase.visualize import view

In [15]:
atoms = Atoms(symbols=['C', 'H', 'H', 'H', 'H'],
            positions=[( 0.0000, 0.0000, 0.0000),
                       ( 0.6533, 0.6533, 0.6533),
                       (-0.6533, 0.6533, 0.6533),
                       ( 0.6533, -0.6533, 0.6533),
                       ( 0.6533, 0.6533, -0.6533)])

In [16]:
atoms.calc = Fireball(charges_method='mdp', options='molecule')

In [18]:
view(atoms, viewer='x3d')

In [19]:
atoms.get_potential_energy()

-211.70762599440044

In [20]:
from ase.optimize import LBFGS
opt = LBFGS(atoms, trajectory='opt.traj', logfile='opt.log')
opt.run(fmax=0.05)

True

In [21]:
read_traj = ase.io.trajectory.TrajectoryReader(traj_file)
view(read_traj)

NameError: name 'ase' is not defined

In [2]:
!pip install nglview
import nglview as nv


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Defaulting to user installation because normal site-packages is not writeable


ModuleNotFoundError: No module named 'nglview'

In [6]:
import pytraj as pt
traj = pt.load('opt.traj')

ModuleNotFoundError: No module named 'pytraj'